<strong>In previous notebook we have generated email dataset now we shall prepare the dataset  as  per our requirement</strong>

In [1]:
from google.colab import drive
drive.mount('gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at gdrive


In [0]:
import os
import cv2
import json
import re
import shutil
import numpy as np
import tarfile
import pickle
from bs4 import BeautifulSoup
import sys
import joblib
from functools import reduce
import operator
import multiprocessing
import pandas as pd
import matplotlib.pyplot as plt
import random
import matplotlib.pyplot as plt
from matplotlib import patches
from itertools import chain
import datetime
from tqdm import tqdm
from zipfile import ZipFile
from google.colab.patches import cv2_imshow
import collections
from collections import Counter
from sklearn.model_selection import train_test_split

from sklearn.utils import shuffle
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from pathlib import Path
from skimage.io import imread
from skimage.color import label2rgb

In [3]:
final_data_points = pd.read_csv('gdrive/My Drive/google/final_data_processed_my_emails.csv')
final_data_points.shape

(164, 6)

In [4]:
final_data_points

,File_name,To,Subject,Previous_email,Content,Type
0,emails_/yernagulahemanth/inbox/299.txt,yernagulahemanth,git hub,NaN,man please go to this link and try to download...,c__d
1,emails_/yernagulahemanth/inbox/412.txt,yernagulahemanth,how are you comminig,NaN,hello mrs yernagulahemanth casestudy are will ...,c__d
2,emails_/yernagulahemanth/inbox/412.txt,hemanthcasestudy4,how are you comminig,hello mrs yernagulahemanth casestudy are will ...,hello yernagulahemanth i will be coming by car...,r__y
3,emails_/yernagulahemanth/inbox/381.txt,yernagulahemanth,regarding deep learning project,NaN,yes this seems fine only three words in the ou...,c__d
4,emails_/yernagulahemanth/inbox/400.txt,yernagulahemanth,webinar,NaN,hello saitejapsk this evening at pm your are h...,c__d
...,...,...,...,...,...,...
159,emails_/yernagulahemanth/sent/117.txt,yernagulahemanth,regarding sql assignment,thank you for your response on sat sep pm appl...,hello team i saw the links i am ok with writin...,r__y
160,emails_/yernagulahemanth/sent/77.txt,team,about final project,NaN,can you explain more about the problem stateme...,c__d
161,emails_/yernagulahemanth/sent/77.txt,yernagulahemanth,about final project,can you explain more about the problem stateme...,it is just like digit recognition we will inpu...,r__y
162,emails_/yernagulahemanth/sent/116.txt,team,regarding sql assignment,NaN,you need to be comfortable with writing nested...,c__d


In [0]:
def data_for_model(index_,dataframe=0):
      '''
      Creates a dataframe such a way that each content  will be divided into x and y 
      if  x is first word of content then y will be  second word of content  i.e if
      x is ith word  then y will be (i+1)th word.Like  this we are going to assign minimum
      one word to maximum 5 words
      '''
      x = []
      y = []
      to = []
      subject = []
      content = []
      file_nm = []
      prev_email = []
      
      type_      = []
      text = dataframe.Content.iloc[index_]
      for i in range(len(text)):
      

              s = 0
              e = i
              for j in range(5):
                      
                      ee = i+1
                      p = ' '.join(text.split()[s:e+1])
                      q = ' '.join(text.split()[ee:ee+j+1])

                      if len(q) == 0:
                        continue
                      else:

                        # print(p,'-->',q)
                        
                        x.append(p)
                        y.append(q)
                        to.append(dataframe.To.iloc[index_])
                        subject.append(dataframe.Subject.iloc[index_])
                        prev_email.append(dataframe.Previous_email.iloc[index_])
                        file_nm.append(dataframe.File_name.iloc[index_])
                        content.append(dataframe.Content.iloc[index_])
                        type_.append(dataframe.Type.iloc[index_])

      data_to_model = pd.DataFrame(columns=['To', 'Subject', 'Previous_email', 'Type','x','y'])

      data_to_model['x']=x
      data_to_model['y']=y   
      data_to_model['To'] = to
      data_to_model['File_nm'] = file_nm
      data_to_model['Subject'] = subject
      data_to_model['Previous_email'] = prev_email
      data_to_model['Content'] = content
      data_to_model['Type'] = type_  
      return data_to_model

In [0]:
def multi_processing(df,nm):
      '''
      Given dataframe is passed to data_to_model  function and saves 
      to joblib  folder  with particular name.
      '''

      data_to_model = pd.DataFrame()
      
      shape_ = df.shape[0]
      current_dataframe = df
      for i in tqdm(range(shape_),position=0):
          data_to_model = data_to_model.append(data_for_model(i,df))
          data_to_model.drop_duplicates()

      joblib.dump(data_to_model,'joblib/sample_'+nm)


In [0]:
for i in tqdm(range(0,164,40),position=0):
     
      multiprocess = multiprocessing.Manager()
      p1   =  multiprocessing.Process(target=multi_processing,args=(final_data_points[i:i+10],str(i+10)))
      p2   =  multiprocessing.Process(target=multi_processing,args=(final_data_points[i+10:i+20],str(i+10)+'_'+str(i+20)))
      p3   =  multiprocessing.Process(target=multi_processing,args=(final_data_points[i+20:i+30],str(i+20)+'_'+str(i+30)))
      p4   =  multiprocessing.Process(target=multi_processing,args=(final_data_points[i+30:i+40],str(i+30)+'_'+str(i+40)))
     
      p1.start()
      p2.start()
      p3.start()
      p4.start()
      
      p1.join()
      p2.join()
      p3.join()
      p4.join()
     

100%|██████████| 10/10 [00:01<00:00,  7.08it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

100%|██████████| 5/5 [00:08<00:00,  1.52s/it]


In [0]:
all_samples = []
for i in tqdm(os.listdir('joblib'),position = 0):
  if os.path.isfile('joblib/'+i):
    all_samples.append(joblib.load('joblib/'+i))

df_after_sample = pd.DataFrame()
df_after_sample = df_after_sample.append([i for i in all_samples])



df_after_sample.reset_index(inplace=True)
df_after_sample.drop(['index'],inplace=True,axis= 1)
df_after_sample

100%|██████████| 20/20 [00:00<00:00, 162.31it/s]


,To,Subject,Previous_email,Type,x,y,File_nm,Content
0,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
1,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
2,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can you,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
3,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can you explain,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
4,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can you explain more,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
...,...,...,...,...,...,...,...,...
38865,yernagulahemanth,doubt regarding hstack,NaN,c__d,hello yernagulahemanth please update your code...,call,emails_/yernagulahemanth/inbox/102.txt,hello yernagulahemanth please update your cod...
38866,yernagulahemanth,doubt regarding hstack,NaN,c__d,hello yernagulahemanth please update your code...,call,emails_/yernagulahemanth/inbox/102.txt,hello yernagulahemanth please update your cod...
38867,yernagulahemanth,doubt regarding hstack,NaN,c__d,hello yernagulahemanth please update your code...,call,emails_/yernagulahemanth/inbox/102.txt,hello yernagulahemanth please update your cod...
38868,yernagulahemanth,doubt regarding hstack,NaN,c__d,hello yernagulahemanth please update your code...,call,emails_/yernagulahemanth/inbox/102.txt,hello yernagulahemanth please update your cod...


<strong>Lets delete duplicates in dataframe</strong>

In [0]:
print('Shape of data befor dropping duplicates:',df_after_sample.shape)
df_after_sample = df_after_sample.drop_duplicates()
print('Shape of data after dropping duplicates:',df_after_sample.shape)

Shape of data befor dropping duplicates: (38870, 8)
Shape of data after dropping duplicates: (37230, 8)


In [0]:
df_after_sample.index = [i for i in range(df_after_sample.shape[0])]
df_after_sample.to_csv('gdrive/My Drive/google/my_df_after_sample.csv',index=False)

In [0]:
df_after_sample = pd.read_csv('gdrive/My Drive/google/my_df_after_sample.csv')

In [0]:
df_after_sample

,To,Subject,Previous_email,Type,x,y,File_nm,Content
0,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
1,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
2,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can you,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
3,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can you explain,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
4,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can you explain more,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
...,...,...,...,...,...,...,...,...
37225,yernagulahemanth,doubt regarding hstack,NaN,c__d,hello yernagulahemanth please update your code...,over the,emails_/yernagulahemanth/inbox/102.txt,hello yernagulahemanth please update your cod...
37226,yernagulahemanth,doubt regarding hstack,NaN,c__d,hello yernagulahemanth please update your code...,over the call,emails_/yernagulahemanth/inbox/102.txt,hello yernagulahemanth please update your cod...
37227,yernagulahemanth,doubt regarding hstack,NaN,c__d,hello yernagulahemanth please update your code...,the,emails_/yernagulahemanth/inbox/102.txt,hello yernagulahemanth please update your cod...
37228,yernagulahemanth,doubt regarding hstack,NaN,c__d,hello yernagulahemanth please update your code...,the call,emails_/yernagulahemanth/inbox/102.txt,hello yernagulahemanth please update your cod...


In [0]:
data = pd.read_csv('gdrive/My Drive/google/my_df_after_sample.csv')


In [0]:
data.head()

,To,Subject,Previous_email,Type,x,y,File_nm,Content
0,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
1,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
2,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can you,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
3,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can you explain,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...
4,yernagulahemanth,about final project,NaN,c__d,hello,yernagulahemanth can you explain more,emails_/yernagulahemanth/inbox/185.txt,hello yernagulahemanth can you explain more a...


<strong>Since our main goal is to predict next word(s) when to,sunject,previous email(if any) some  part of content is given so lets featurize in following way</strong>

<table>
    <tr>
      <th>Sentance</th>
      <th>Output</th>
    </tr>
    <tr>
      <td> This </td>
      <td> is</td>
    </tr>
    <tr>
      <td> This </td>
      <td> is introduction </td>
    </tr>
    <tr>
      <td> This </td>
      <td> is introduction to</td>
    </tr>
    <tr>
      <td> This </td>
      <td> is introduction to my</td>
    </tr>
    <tr>
      <td> This </td>
      <td> is introduction to my project</td>
    </tr>
    <tr>
      <td> This is </td>
      <td> introduction</td>
    </tr>
    <tr>
      <td> This is</td>
      <td> introduction to </td>
    </tr>
    <tr>
      <td> This is </td>
      <td> introduction to my</td>
    </tr>
    <tr>
      <td> This is</td>
      <td> introduction to my</td>
    </tr>
    <tr>
      <td> This is</td>
      <td> introduction to my project</td>
    </tr>
    <tr>
      <td> This is introduction</td>
      <td> to</td>
    </tr>
    <tr>
      <td> This is introduction</td>
      <td>  to my </td>
    </tr>
    <tr>
      <td> This is introduction </td>
      <td> to my project</td>
    </tr>
    
   
 </table>

 <strong>Note:</strong><p>Each part is seperated with their tags like < to >< sub >< prv >< cont ></p>

In [0]:
tspc = [] #  combination of To, Subject, Previous Email(if any), content of email

for i in range(data.shape[0]):
  tspc.append('<to> ' + str(data.To.iloc[i])+' <prv> '+ str(data.Previous_email.iloc[i])+' <sub> ' +data.Subject.iloc[i] + ' <cont> ' +  data.x.iloc[i])



In [0]:
final_data = pd.DataFrame(zip(tspc,data.y.values),columns=['x','y'])
final_data.head()

,x,y
0,<to> yernagulahemanth <prv> nan <sub> about f...,yernagulahemanth
1,<to> yernagulahemanth <prv> nan <sub> about f...,yernagulahemanth can
2,<to> yernagulahemanth <prv> nan <sub> about f...,yernagulahemanth can you
3,<to> yernagulahemanth <prv> nan <sub> about f...,yernagulahemanth can you explain
4,<to> yernagulahemanth <prv> nan <sub> about f...,yernagulahemanth can you explain more


In [0]:
final_data.to_csv('gdrive/My Drive/google/final_data_my_emails.csv',index=False)
final_data.shape

(37230, 2)

<strong>Lets apply models in next document</strong>